#Naver Movie review

In [ ]:
!pip3 install wget

In [ ]:
import wget
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
import pandas as pd
train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df = train_df.dropna()

In [ ]:
# !pip3 install konlpy

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from konlpy.tag import Okt
# okt = Okt()
# okt.morphs('아 더빙.. 진짜 짜증나네요. 목소리')
# tfidfVectorizer = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
# tfidVec_matrix_train = tfidfVectorizer.fit_transform(train_df['document'])

In [ ]:
# tfidVec_matrix_train

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['document']);		

In [ ]:
# tokenizer.word_index

In [ ]:
x_train_all = tokenizer.texts_to_sequences(train_df['document'])
type(x_train_all)

In [ ]:
max(len(item) for item in x_train_all), min(len(item) for item in x_train_all)

In [ ]:
x_train = [item for item in x_train_all if len(item) > 0]
min(len(item) for item in x_train), len(x_train)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
pad_len = 60//2
x_train = pad_sequences(x_train, maxlen=pad_len)		# pad 0 As 60//2

In [ ]:
type(x_train), len(x_train), len(train_df['label']), len(tokenizer.word_counts)

In [ ]:
y_train = train_df['label'][:len(x_train)].values
len(y_train), len(train_df['label'][:149755]), type(train_df['label'][:149755].values)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Embedding(len(tokenizer.word_counts), 24),
    layers.Bidirectional(layers.LSTM(64)),
    # layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    # layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    # layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
model.input

In [ ]:
hist = model.fit(x_train, y_train, epochs=1,
                    # validation_split=0.3, 
                    # validation_steps=30, batch_size=256
                    )

In [ ]:
pred_text = '방금 보고 옴. mvp: 잉그리스 여왕(혼자서 4인분 능가하는 캐리력)트롤러: 오로라(웃으면서 맘 놓게 하고 뒤통수 여러번 침, 도라이)무임탑승자: 존 왕(숙면하고 나니 염원하던 화합 이뤄짐 대박 버스)말레피센트 개고생 시키고 기어이 마지막에 한번 더 구해줬더니 오로라가 쳐웃으면서 well, well 하는 거 보고 도라이 아냐? 라고 소리내서 말함'
pred_text = tokenizer.texts_to_sequences([pred_text])
print(pred_textt)
-

#IMDB review

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [ ]:
encoder = info.features['text'].encoder

In [ ]:
# encoder.vocab_size, encoder.subwords

In [ ]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

##Prepare the data for training

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
# train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = train_dataset.shuffle(BUFFER_SIZE)
test_dataset = test_dataset.padded_batch(BATCH_SIZE)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Embedding(encoder.vocab_size, 64),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# model.input

In [ ]:
hist = model.fit(train_dataset, epochs=2,
                    # validation_split=0.3,       # Error without padded_batch(BATCH_SIZE)
                    # validation_data=test_dataset, 
                    # validation_steps=30, batch_size=256
                    )

In [ ]:
pred_text = ('The movie was cool. The animation and the graphics '
                    'were out of this world. I would recommend this movie.')
pred_text = encoder.encode(pred_text)
# pred_text = pred_text.padded_batch(BATCH_SIZE)
pred = model.predict(tf.expand_dims(pred_text, axis=0))   #Returns a tensor with an additional dimension inserted at index axis.
pred

In [ ]:
pred_text